In [39]:
import psycopg2
import pandas as pd

# Prepare the Data

Run `docker-compose up` to get Postgres up and running.

In [40]:
credentials = "dbname='astronomy' user='user' host='localhost' password='pass'"
conn = psycopg2.connect(credentials)
cur = conn.cursor()

cur.execute("""
CREATE TABLE Star (
  kepler_id INTEGER NOT NULL,
  koi_name VARCHAR(20) NOT NULL,
  t_eff INTEGER,
  radius FLOAT(5),
  PRIMARY KEY (koi_name)
);


COPY Star (kepler_id, koi_name, t_eff, radius)
  FROM '/data/lesson3/stars.csv' CSV;

CREATE TABLE Planet (
  kepler_id INTEGER NOT NULL,
  koi_name VARCHAR(20) NOT NULL,
  kepler_name VARCHAR(20),
  status VARCHAR(20) NOT NULL,
  period FLOAT,
  radius FLOAT,
  t_eq INTEGER,
  PRIMARY KEY (koi_name)
);


COPY Planet (kepler_id, koi_name, kepler_name, status, period, radius, t_eq)
  FROM '/data/lesson3/planets.csv' CSV;
""")

# Task 1
Write an SQL query to find the radius and temperature of the stars in the
Star table that are larger than our sun.

In [41]:
pd.read_sql("""
SELECT radius, t_eff FROM Star WHERE radius > 1;
""", conn)

,radius,t_eff
0,3.523,6335
1,1.965,8782
2,1.032,6319
3,27.384,3789


# Task 2
Write a range query which returns the `kepler_id` and the `t_eff` attributes
of all those stars in the Star table whose temperature lies between 5000 and
6000 Kelvin (inclusive)

In [42]:
pd.read_sql("""
SELECT kepler_id, t_eff FROM Star WHERE t_eff BETWEEN 5000 AND 6000;
""", conn)

,kepler_id,t_eff
0,3836450,5160
1,6590362,5926
2,8106973,5810


# Task 3
Write a query to find the `kepler_name` and `radius` of each planet in the
`Planet` table which is a confirmed exoplanet, meaning that their `kepler_name`
is not `NULL`, or, equivalently, whose status is `'CONFIRMED'`.

Restrict your results to those planets whose radius lies between one and three
earth radii, and remember that the radius of the planets is relative to the
earth radius.

In [43]:
pd.read_sql("""
SELECT kepler_name, radius FROM Planet
  WHERE kepler_name IS NOT NULL AND radius BETWEEN 1 AND 3;
""", conn)

,kepler_name,radius
0,Kepler-10 b,1.45
1,Kepler-106 c,2.35
2,Kepler-52 d,1.80
3,Kepler-239 b,2.36
4,Kepler-239 c,2.19


# Tear down

In [44]:
cur.close()
conn.close()